In [1]:
GENIUS_API_TOKEN='F2SBCBsZrWPf0Isy6wKwnYyVDpZF9z67Zzd_UjHPNvEylkr0j6D0Qo0sQMeCGfW9'

In [2]:
import requests
from bs4 import BeautifulSoup
import os
import re
import pickle
import numpy as np

In [3]:
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url) 
        if (len(songs) == song_cap):
            break
        else:
            page += 1
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs
    
request_song_url('Lana Del Rey', 2)

Found 2 songs by Lana Del Rey


['https://genius.com/Lana-del-rey-young-and-beautiful-lyrics',
 'https://genius.com/Lana-del-rey-summertime-sadness-lyrics']

In [4]:
def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    for tag in html.select('div[class^="Lyrics__Container"], .song_body-lyrics p'):
        lyrics = tag.get_text(strip=True, separator='\n')
        if lyrics:
            print(lyrics)
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])         
    return lyrics
print(scrape_song_lyrics('https://genius.com/Lana-del-rey-young-and-beautiful-lyrics'))

[Verse 1]
I've seen the world, done it all
Had my cake now
Diamonds, brilliant,
and Bel Air now
Hot summer nights, mid-July
When you and I were forever wild
The crazy days, city lights
The way you'd play with me like a child
[Chorus]
Will you still love me when I'm no longer young and beautiful?
Will you still love me when I got nothing but my aching soul?
I know you will, I know you will, I know that you will
Will you still love me when I'm no longer beautiful?
[Verse 2]
I've seen the world, lit it up as my stage now
Channeling angels in the new age now
Hot summer days
, rock and roll
The way you'd play for me at your show
And all the ways I got to know
Your pretty face and electric soul
[Chorus]
Will you still love me when I'm no longer young and beautiful?
Will you still love me when I got nothing but my aching soul?
I know you will, I know you will, I know that you will
Will you still love me when I'm no longer beautiful?
[Bridge]
Dear Lord, when I get to heaven
Please, let me brin

In [5]:
lyrs = []
def write_lyrics_to_file(artist_name, song_count):
    try:
        f = open('lyrics/' + artist_name.lower() + '.txt', 'wb')
    except:
        print('e')
    urls = request_song_url(artist_name, song_count)
    for url in urls:
        lyrics = scrape_song_lyrics(url)
        lyrs.append(lyrics.encode("utf8"))
        f.write(lyrics.encode("utf8"))
    f.close()
    num_lines = sum(1 for line in open('lyrics/' + artist_name.lower() + '.txt', 'rb'))
    print('Wrote {} lines to file from {} songs'.format(num_lines, song_count))

In [6]:
write_lyrics_to_file('Kendrick Lamar', 1)

Found 1 songs by Kendrick Lamar
[Produced by Hit-Boy]
[Intro]
Uh, Martin had a dream
Martin had a dream
Kendrick have a dream
[Chorus]
All my life I want money and power
Respect my mind or die from lead shower
I pray my dick get big as the Eiffel Tower
So I can fuck the world for seventy-two hours
[Verse 1]
Goddamn I feel amazin', damn, I'm in the Matrix
My mind is livin' on cloud nine and this 9 is never on vacation
Start up that Maserati and – vroom-vroom! – I'm racin'
Poppin' pills in the lobby and I pray they don't find her naked
And I pray you niggas is hatin',
shooters go after Judas
Jesus Christ, if I live life on my knees ain't no need to do this
Park it in front of Lueders, next to that Church's Chicken
All you pussies is losers, all my niggas is winners, screamin'—
[Chorus]
All my life I want money and power
Respect my mind or die from lead shower
I pray my dick get big as the Eiffel Tower
So I can fuck the world for seventy-two hours
[Post-Chorus 1]
Goddamn I got bitches, da

In [7]:
lyrs

[b'Let it run, Ali\nMartin had a dream\nMartin had a dream\nKendrick have a dream']

In [8]:
infile = open('namelist-from-genius.pkl','rb')
artist_list = pickle.load(infile)
infile.close()

In [9]:
artist_list_np = np.array(artist_list)
artist_list_np

array(['A1 x J', 'A2', 'A2', ..., '9th Stree', '9th Wonde',
       '9 to 5 Original Broadway Ca'], dtype='<U225')

In [10]:
for name in artist_list_np:
    print(name)
    write_lyrics_to_file(str(name), 1)